In [4]:
import os

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import SimpleITK as sitk
from tqdm import tqdm

# from src.seg_dataset import SegDatasetCreator
from src.utils.competition_metric import ULS23_evaluator

import collections
import itertools

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from scipy.ndimage import binary_erosion, label
from scipy.spatial.distance import pdist, squareform

import pandas as pd
from collections import Counter
import random
import glob

from tqdm import tqdm

# Enable tqdm for pandas
tqdm.pandas()


In [6]:
class NiiSliceDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        # List all image slices
        self.image_paths = sorted(glob(os.path.join(images_dir, "MIX_*_*.nii.gz")))
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]

        # Extract ID, e.g., MIX_00001_0000.nii.gz -> MIX_00001
        basename = os.path.basename(image_path)
        id_part = "_".join(basename.split("_")[:2])  # MIX_00001

        label_path = os.path.join(self.labels_dir, f"{id_part}.nii.gz")

        # Read image slice (single slice)
        image_itk = sitk.ReadImage(image_path)
        image = sitk.GetArrayFromImage(image_itk).astype(np.float32)

        # Read full label volume
        label_itk = sitk.ReadImage(label_path)
        label = sitk.GetArrayFromImage(label_itk).astype(np.int64)

        if self.transform:
            image, label = self.transform(image, label)

        # Convert to torch tensors
        image_tensor = torch.from_numpy(image) 
        label_tensor = torch.from_numpy(label) 

        return image_tensor, label_tensor


test_data_path = "/d/hpc/home/jf73497/projects/aimi-project-data/raw/Dataset001_MIX"

test_dataset = NiiSliceDataset(test_data_path + "/imagesTs", test_data_path + "/labelsTs")

TypeError: 'module' object is not callable

In [20]:
def calculate_lesion_sizes(label_path, spacings_path):
    #print(f"Calculating lesion sizes for {label_path} and {spacings_path}")
    label_array = torch.load(label_path, weights_only=False)
    # Spacing should be reversed in order to align the z axis in the image and spacings arrays but w/e  
    voxel_spacing = np.array(torch.load(spacings_path, weights_only=False)[:3])
    #print(f"Voxel spacing: {voxel_spacing}")

    #print(f"Label array shape: {label_array.shape}") 
    label_array = (label_array[1:, ...]).squeeze()
    #print(f"Label array shape after removing first slice: {label_array.shape}") 
    _, _, long_axis_points, short_axis_points = evaluator.long_and_short_axis_diameters(label_array)

    #print(f"Long axis points: {long_axis_points}")
    if long_axis_points is None:
        # No idea why this happens but its only a few cases and i think we can ignore it
        return (0, 0)
    
    p1_long, p2_long = long_axis_points
    p1_short, p2_short = short_axis_points

    vec_long = (p2_long - p1_long) * voxel_spacing
    vec_short = (p2_short - p1_short) * voxel_spacing

    # Compute Euclidean distances (in mm)
    label_long_mm = np.linalg.norm(vec_long)
    label_short_mm = np.linalg.norm(vec_short)
    
    return (label_long_mm, label_short_mm)